In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [ ]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--jars mongo-hadoop-spark.jar --driver-class-path mongo-hadoop-spark.jar"

In [2]:
import optimus as op

Just checking that all necessary environment vars are present...
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\java8
-----
Starting or getting SparkSession and SparkContext...
Setting checkpoint folder (local). If you are in a cluster change it with set_check_point_ folder(path,'hadoop').
Deleting previous folder if exists...
Creation of checkpoint directory...
Done.


In [3]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, IntegerType, ArrayType

df = op.create_df([
                ("  I like     fish  ", 1, "dog", "housé" ),
                ("    zombies", 2, "cat", "tv"),
                ("simpsons   cat lady", 2, "frog", "table"),
                (None, 3, "eagle", "glass")
            ],
            [
                ("words", StringType(), True),
                ("num", IntegerType(), True),
                ("animals", StringType(), True),
                ("thing", StringType(), True)
            ])
df.show()
df = df.upper("words").lower("words").reverse("words").trim("words").remove_accents("words")

+-------------------+---+-------+-----+
|              words|num|animals|thing|
+-------------------+---+-------+-----+
|  I like     fish  |  1|    dog|housé|
|            zombies|  2|    cat|   tv|
|simpsons   cat lady|  2|   frog|table|
|               null|  3|  eagle|glass|
+-------------------+---+-------+-----+



TypeError: normalize() argument 2 must be str, not Column

In [4]:
df.columns

['words', 'num', 'animals', 'thing']

In [14]:
df

DataFrame[words: string, num: int, animals: string, thing: string]

In [4]:
df= df.astype([("num","string")])

In [5]:
df

DataFrame[num: string, words: string, animals: string, thing: string]

In [7]:
df.move_col("num", "thing", "after")

DataFrame[words: string, animals: string, thing: string, num: string]

In [15]:
df.show()

+-------------------+---+-------+-----+
|              words|num|animals|thing|
+-------------------+---+-------+-----+
|  I like     fish  |  1|    dog|housé|
|            zombies|  2|    cat|   tv|
|simpsons   cat lady|  2|   frog|table|
|               null|  3|  eagle|glass|
+-------------------+---+-------+-----+



In [95]:
tup = [("num","int")]

columns = []
new_col = ""
for c in df.columns:
    for t in tup:
        if t[0] == c:
            new_col = col(c).cast(DICT_TYPES[TYPES[t[1]]]).alias(c)
        else:
            new_col = col(c)
    columns.append(new_col)

#print(DICT_TYPES[TYPES["string"]])
df =df.select(columns)
df


DataFrame[words: string, num: int, animals: string, thing: string]

In [80]:
df

DataFrame[words: string, num: int, animals: string, thing: string]

In [23]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DoubleType

# You can use string, str or String as param
TYPES = {'string': 'string', 'str': 'string', 'String': 'string', 'integer': 'int',
         'int': 'int', 'float': 'float', 'double': 'double', 'Double': 'double'}

# Instead StringType() just use string
DICT_TYPES = {'string': StringType(), 'int': IntegerType(), 'float': FloatType(), 'double': DoubleType()}



def a(self, cols_and_types):
    """

    :param self:
    :param cols_and_types:
            List of tuples of column names and types to be casted. This variable should have the
            following structure:

            colsAndTypes = [('columnName1', 'integer'), ('columnName2', 'float'), ('columnName3', 'string')]

            The first parameter in each tuple is the column name, the second is the final datatype of column after
            the transformation is made.
    :return:
    """

    # Check if columnNames to be process are in dataframe
    column_names = self._parse_columns(cols_and_types, 0)

    not_specified_columns = filter(lambda c: c not in column_names, self.columns)

    exprs = [col(column[0]).cast(DICT_TYPES[TYPES[column[1]]]).alias(column[0]) for column in cols_and_types] + [
        col(column) for column in not_specified_columns]
    
    
    for 
    print(exprs)
    #return self.select(*exprs)

df.a = a

df.a(df, [("num","int")])
df.show()

[Column<b'CAST(num AS INT) AS `num`'>, Column<b'words'>, Column<b'animals'>, Column<b'thing'>]
+-------------------+---+-------+-----+
|              words|num|animals|thing|
+-------------------+---+-------+-----+
|  I like     fish  |  1|    dog|housé|
|            zombies|  2|    cat|   tv|
|simpsons   cat lady|  2|   frog|table|
|               null|  3|  eagle|glass|
+-------------------+---+-------+-----+



In [17]:
df = df.drop_("animals")
df.show()

+-------------------+---+-----+
|              words|num|thing|
+-------------------+---+-----+
|  I like     fish  |  1|housé|
|            zombies|  2|   tv|
|simpsons   cat lady|  2|table|
|               null|  3|glass|
+-------------------+---+-----+



In [ ]:
columns = ([("num", "numbers")])
for c in columns: 
    assert isinstance(c, tuple)
    
df = df.rename(columns)    
df.show()

In [7]:
print(df.min(["numbers", "numbers"]))
print(df.max("numbers"))
print(df.range("numbers"))
print(df.median("numbers"))
print(df.stddev("numbers"))
print(df.kurt("numbers"))
print(df.mean("numbers"))
print(df.skewness("numbers"))
print(df.sum("numbers"))
print(df.variance("numbers"))

[1, 1]
3
[2]
2.0
0.816496580927726
-1.0000000000000002
2.0
0.0
0.0
0.6666666666666666


In [13]:
spark = op.get_spark()
sc = spark.sparkContext

In [59]:
import json
j = {'abc':{"a":1, "b":2}, 'def':2, 'ghi':3}
a=[json.dumps(j)]
jsonRDD = sc.parallelize(a)

df = spark.read.json(jsonRDD)
df.show()

+------+---+---+
|   abc|def|ghi|
+------+---+---+
|[1, 2]|  2|  3|
+------+---+---+



In [55]:
firstDF = spark.range(3).toDF("myCol")
newRow = spark.createDataFrame([[20]])
appended = firstDF.union(newRow)
appended.show()

+-----+
|myCol|
+-----+
|    0|
|    1|
|    2|
|   20|
+-----+



In [2]:
df.show()

NameError: name 'df' is not defined

In [31]:
df
df.show()
from pyspark.sql.functions import when  

def replace(df, columns, to_replace, value):
    for r in to_replace:
        print(r)
        newsdf = df.replace(to_replace, value, columns)
        #newsdf = df.withColumn(column, when(df[column] == r, value).otherwise(0))
    newsdf.show()
        
replace(df,["num","words"], [0, 1, 2, 3], "4")    
df


+-------------------+---+-------+-----+
|              words|num|animals|thing|
+-------------------+---+-------+-----+
|  I like     fish  |  1|    dog|housé|
|            zombies|  2|    cat|   tv|
|simpsons   cat lady|  2|   frog|table|
|               null|  3|  eagle|glass|
+-------------------+---+-------+-----+

0


ValueError: Mixed type replacements are not supported

In [38]:
a = df[df.num == '3']
a.show()
df[1].show()

+-----+---+-------+-----+
|words|num|animals|thing|
+-----+---+-------+-----+
| null|  3|  eagle|glass|
+-----+---+-------+-----+



TypeError: 'Column' object is not callable